In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ast

from os import listdir
from os.path import isfile, join


In [2]:
# EXTRACT SENTIMENT AGGREGATED BY TIMESPAN
ticker = 'AAPL'
mypath = '/home/simone/Desktop/NLFF/Word2Vec/conceptsDataset/summary/'

tickFiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

news =  pd.read_csv(mypath+str(ticker)+'.csv')
news = news.drop(['Unnamed: 0'], axis=1)
news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['DATE']]
#news['entSentVec'] = [ast.literal_eval(row) for row in news['entSentVec']]

print(list(news.columns))


['TICKER', 'FIGI_TICKER', 'FIGI', 'TITLE', 'DATE', 'URL', 'SUMMARY', 'concepts', 'polarity', 'attention', 'pleasantness', 'aptitude', 'sensitivity', 'stemmed_polarity', 'stemmed_attention', 'stemmed_pleasantness', 'stemmed_aptitude', 'stemmed_sensitivity', 'stemmed_concepts']


In [3]:


timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+str(ticker)+".csv")
timeSpan = timeSpan['date'].tolist()
# This dataset is already GMT+0
timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]

features_names = ['polarity','attention','pleasantness','aptitude','sensitivity','stemmed_polarity','stemmed_attention','stemmed_pleasantness','stemmed_aptitude','stemmed_sensitivity']

sentiment = pd.DataFrame(columns=['initTime']+features_names)
#print(list(sentiment.columns))

#Prendiamo intervallo massimo dove ci sono entrambi i dataset:
#massimo della data iniziale tra i due
#minimo della data finale tra i due
initDate = max(timeSpan[0], news['DATE'][0])
finalDate = min(timeSpan[len(timeSpan)-1], news['DATE'][len(news)-1])



i = 0
j = 0
# ALLINEAMENTO INIZIO
while(timeSpan[j] < initDate):
    j+=1
init = j
while(news['DATE'][i] <= initDate - timedelta(hours=1)):
    i+=1

print(timeSpan[j])
print(news['DATE'][i])
    
print("Start: "+str(initDate) +"("+str(timeSpan[j])+") i: "+str(i)+" j: "+str(j))
print("End: "+str(finalDate))

#ALLINEAMENTO FINE DENTRO I WHILE
while( news['DATE'][i] < finalDate and timeSpan[j] < finalDate ):
    initTime = timeSpan[j]
    normal_sum = {feature_name: 0 for feature_name in features_names}    
    num_tot_news = 0
    while(i<len(news)-1 and timeSpan[j] > news['DATE'][i]):
        if(i%1000 == 0):
            print(i)
        if not (timeSpan[j] > news['DATE'][i] and timeSpan[j-1] <= news['DATE'][i]):
            print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
            assert False
#         a = news['entSentVec'][i]
#         print(type(a))
#         print(len(a))
#         a = np.asarray(news['entSentVec'][i])
#         print(type(a))
#         print(a.shape)
            
        normal_sum = {f : normal_sum[f]+news[f][i] for f in features_names}        
        num_tot_news +=1
        i+=1
    j+=1
    #sentiment.loc[j] = {'initTime':initTime, 'num_news':num_tot_news}
    sentiment.loc[j,'initTime'] = initTime
    sentiment.loc[j,'num_news'] = num_tot_news
    if num_tot_news>0:
        normal_sum = {f : normal_sum[f]/num_tot_news for f in features_names}        
    for f in normal_sum.keys():
        sentiment.loc[j,f] = normal_sum[f]
    


end = j
if(len(sentiment) != end - init):
    print(len(sentiment))
    print(end - init + 1)
    assert False


sentiment.to_csv('Aggregated_senticnet_dataset/'+str(ticker)+'_senticnet.csv')

2017-04-03 18:00:00
2017-04-03 17:27:34
Start: 2017-04-03 17:27:34(2017-04-03 18:00:00) i: 0 j: 3817
End: 2018-06-21 23:20:00
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [4]:
sentiment = sentiment.sort_values(by=['initTime'])
sentiment = sentiment.reset_index(drop=True)
sentiment.head()

,initTime,polarity,attention,pleasantness,aptitude,sensitivity,stemmed_polarity,stemmed_attention,stemmed_pleasantness,stemmed_aptitude,stemmed_sensitivity,num_news
0,2017-04-03 18:00:00,0.255305,0.193205,0.156891,0.190928,0.0641241,0.330112,0.176294,0.214781,0.309051,0.0556677,7.0
1,2017-04-03 19:00:00,0.201035,0.0182974,0.141537,0.17452,-0.0313478,0.202393,0.0883699,0.119848,0.186554,-0.0391006,6.0
2,2017-04-03 20:00:00,0.267883,0.117938,0.174592,0.212881,0.00211711,0.109875,0.202533,0.0129044,0.0499178,-0.0818073,8.0
3,2017-04-04 14:00:00,0.194742,0.0715313,0.141741,0.147521,0.0251548,0.242885,0.0295263,0.183921,0.195147,0.0205663,34.0
4,2017-04-04 15:00:00,0,0,0,0,0,0,0,0,0,0,0.0


In [5]:
sentiment.to_csv('Aggregated_senticnet_dataset/'+str(ticker)+'_senticnet.csv')